In [432]:
import re
import json
import time
import pickle
from collections import defaultdict

import spacy
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [438]:
%%time
OPTIONS = Options()
OPTIONS.add_argument('--headless')
# make sure Homebrew is installed
# then run 'brew tap homebrew/cask && brew cask install chromedriver'
driver = webdriver.Chrome(options=OPTIONS)

CPU times: user 4.42 ms, sys: 9.63 ms, total: 14 ms
Wall time: 1.26 s


In [70]:
# python3 -m spacy download en
nlp = spacy.load('en')

def tokenize(line):
    return [(token.text, token.tag_) for token in nlp(line)]

In [71]:
def numerical(line):
    # replace everything to '' except whitespace, alphanumeric character
    line = re.sub(r'[^\w\s]', '', line)
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        # if the word is not numerical
        if not pair[1] == "CD":
            return False
    return True

def nouns_only(line):
    # replace everything to '' except whitespace, alphanumeric character
    line = re.sub(r'[^\w\s]', '', line)
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        # if the word is not a noun or cardinal number
        if not (pair[1] == "NN" or pair[1] == "NNS"):
            return False
    return True

In [72]:
type_exceptions = ['can', 'tablespoon']

for exception in type_exceptions:
    print(tokenize(exception), spacy.explain(tokenize(exception)[0][1]))

[('can', 'MD')] verb, modal auxiliary
[('tablespoon', 'VB')] verb, base form


In [405]:
def extract_quantity_in_backets(line):
    # find '(abc)' where 'abc' is in arbitrary length and 'abc' does not contain brackets
    pattern = re.compile(r'\([^\(\)]*\)') 
    match = re.findall(pattern, line)
    if len(match) != 0:
        if line.split()[1] not in match[0]:
            return None
        return match
    
def extract_preparation(line):
    # find ', abc' where 'abc' is in arbitrary length
    pattern = re.compile(r'\b, [^\(\)]*')
    match = re.findall(pattern, line)
    if len(match) != 0:
        return match
    
def extract_all(line):    
    type_exceptions = ['can', 'tablespoon', 'oz']
    quantity_split = []
    measurement = None
    
    # extract preparation
    line = line.replace(' -', ',')
    preparation = extract_preparation(line)
    if preparation:
#         print('preparation[0]:', preparation[0])
#         print(line)
        line = re.sub(r'{0}'.format(preparation[0]), '', line)
        # remove ', ' prefix
        preparation = preparation[0][2:].strip()
        
    # extract quantity in backets
    quantity_in_brackets = extract_quantity_in_backets(line)
    if quantity_in_brackets:
        line = re.sub(r'\({0}\)'.format(quantity_in_brackets[0]), '', line)
        quantity_in_brackets = quantity_in_brackets[0]
    
    line_split = line.split()
    # extract quantity from the first word if the word contains a digit
    if any(char.isdigit() for char in line_split[0]):
        quantity_split.append(line_split[0])
    
        # extract quantity from the second word if the word contains a digit
        if any(char.isdigit() for char in line_split[1]):
            quantity_split.append(line_split[1])
            # check measurement type
            # to avoid case like '1 large tomato, seeded and chopped'
            if nouns_only(line_split[2]) or line_split[2] in type_exceptions:
                measurement = line_split[2]
        else:
            # check line_split length for case like '1 egg' or '1/2 onion, chopped'
            if len(line_split) > 2 and (nouns_only(line_split[1]) or line_split[1] in type_exceptions):
                measurement = line_split[1]
        line = re.sub(r'{0}'.format(' '.join(quantity_split)), '', line)
    
    if measurement:
        line = re.sub(r'{0}'.format(measurement), '', line)
    
    # append quantity in backets at the end
    if quantity_in_brackets:
        quantity_split.append(quantity_in_brackets)
    quantity = ' '.join(quantity_split)

    # extract ingredient name
    line = re.sub(r'[ ]?®', '', line)
    line = re.sub(r'[ ]?™', '', line)
    ingredient_name = line.strip()
    
    # if 'or to taske' or 'or as needed' in preparation
    if preparation is not None and 'or ' in preparation:
        quantity += ' ' + preparation
        preparation = None    
    
    return quantity, measurement, ingredient_name, preparation

In [406]:
# test on tricky cases
# extract_all('4 roma (plum) tomatoes, chopped')
# extract_all('1 1/2 pounds colossal shrimp, EZ-peel type (deveined and shells split down the back)')
# extract_all('1 clove garlic, minced, or more to taste')
# extract_all('4 pounds Korean-style short ribs (beef chuck flanken, cut 1/3 to 1/2 inch thick across bones)')
# extract_all('1/2 (14 ounce) package vegetarian ground beef (e.g., Gimme Lean TM)')
# extract_all('1 0.42 oz packet concentrated vegetable base (such as Swanson® Vegetable Flavor Boost®) (optional)')
extract_all('2 avocados - peeled, pitted, and sliced')

('2', None, 'avocados', 'peeled, pitted, and sliced')

In [313]:
def extract_descriptor(ingredient_name):
    type_exceptions = ['parsley', 'garlic', 'chili']
    descriptor = []
    token_tag_pairs = []
    
    for element in ingredient_name.split():
        # treat compound word with hyphen as an adjective
        if '-' in element:
            token_tag_pairs.append((element, 'JJ'))
        else:
            token_tag_pairs.append([(token.text, token.tag_) for token in nlp(element)][0])
    
    for pair in token_tag_pairs:
        # if the word is an adjective, an adverb, or a past participle of a verb, or exception like 'ground'
        if pair[1] == "JJ" or pair[1] == "RB" or pair[1] == "VBN" or pair[0] == 'ground':
            if pair[0] not in type_exceptions:
                descriptor.append(pair[0])
    if len(descriptor) != 0:
        return ' '.join(descriptor)

In [314]:
# test
# extract_descriptor('extra-firm tofu') 
# extract_descriptor('all-purpose flour') 
# extract_descriptor('freshly ground black pepper') 
extract_descriptor('1/2 cup chopped parsley')
extract_descriptor('1 1/2 pounds colossal shrimp, EZ-peel type (deveined and shells split down the back)')

'colossal EZ-peel split back'

In [315]:
def extract_ingredients_nouns(line):
    ingredients_nouns = set()
    token_tag_pairs = tokenize(line)
    for pair in token_tag_pairs:
        if pair[1] == 'NN' or pair[1] == 'NNS':
            ingredients_nouns |= {pair[0]}
    return ingredients_nouns

In [316]:
def get_ingredient_list(url):
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
    global driver
    try:
        driver.get(url)
    except:
        print('relaunching webdriver...')
        driver = webdriver.Chrome(DRIVER_LOCATION, options=OPTIONS)
        driver.get(url)
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # extract ingredients section from the webpage
    ingredients = set([element.label.text.strip() for element in soup.find_all(class_='checkList__line')])
    # remove unnecessary elements
    unnecessary = ['', 'Add all ingredients to list']
    for i in unnecessary:
        if i in ingredients:
            ingredients.remove(i)
    return ingredients

In [305]:
%%time
get_ingredient_list('https://www.allrecipes.com/recipe/246628/spaghetti-cacio-e-pepe/?internalSource=popular&referringContentType=Homepage&clickId=cardslot%203')

relaunching webdriver...
CPU times: user 101 ms, sys: 10.1 ms, total: 111 ms
Wall time: 8.84 s


{'1 3/4 cups grated Pecorino Romano cheese',
 '1 pound spaghetti',
 '2 cloves garlic, minced',
 '2 teaspoons ground black pepper',
 '6 tablespoons olive oil'}

In [306]:
def lemmatize(line):
    if len(line.split()) == 1:
        return [token.lemma_ for token in nlp(line)][0]
    return line

In [408]:
def get_ingredients_from_url(url):
    print(url)
    ingredients_url = set()
    ingredients_measurement_dict_url = defaultdict(set)
    ingredients_descriptor_dict_url = defaultdict(set)
    ingredients_preparation_dict_url = defaultdict(set)
    
    ingredient_list = get_ingredient_list(url)
    for line in ingredient_list:

        
        quantity, measurement, ingredient_name, preparation = extract_all(line)
        # exceptions like "topping:"
        if ':' in line:
            print(line)
            print(url)
            continue
            
        descriptor = extract_descriptor(ingredient_name)
        # remove descriptor if not None
        ingredient = ingredient_name.replace(descriptor, '').strip() if descriptor else ingredient_name
        # if ingredient is empty after removing descriptor
        if ingredient == '':
            ingredient = ingredient_name
        # remove ' to taste' in ingredient if any
        ingredient = re.sub(r'(or)? to taste', '', ingredient)
        ingredient = ' '.join(ingredient.split())
        # lemmatize ingredient
#         ingredient = lemmatize(ingredient)
        
        if ingredient not in ingredients_measurement_dict_url:
            ingredients_measurement_dict_url[ingredient] |= {measurement}
        if ingredient not in ingredients_descriptor_dict_url:
            ingredients_descriptor_dict_url[ingredient] |= {descriptor}
        if ingredient not in ingredients_preparation_dict_url:
            ingredients_preparation_dict_url[ingredient] |= {preparation}
        ingredients_url |= {ingredient}
        
        # test
        print(line)
        print('quantity:', quantity)
        print('measurement:', measurement)
        print('ingredient name:', ingredient_name)
        print('descriptor:', descriptor)
        print('ingredient without descriptor:', ingredient)
        print('preparation:', preparation)
        print()
    
    return ingredients_url, ingredients_measurement_dict_url, ingredients_descriptor_dict_url, ingredients_preparation_dict_url

In [389]:
def get_recipe_urls(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    recipe_cards = soup.find_all('div', class_='fixed-recipe-card__info')
    recipe_urls = []
    for recipe_card in recipe_cards:
        url = recipe_card.find('a')['href']
        recipe_urls.append(url)
    return recipe_urls

In [420]:
def get_all_ingredients_in_category(category_url):
    ingredients_freq_dict_all = dict()
    ingredients_measurement_dict_all = defaultdict(set)
    ingredients_descriptor_dict_all = defaultdict(set)
    ingredients_preparation_dict_all = defaultdict(set)
    
    recipe_urls = get_recipe_urls(category_url)
    for recipe_url in recipe_urls:
        recipe_ingredients, recipe_ingredients_measurement_dict, ingredients_descriptor_dict, ingredients_preparation_dict = get_ingredients_from_url(recipe_url)
        for i in recipe_ingredients:
            if i in ingredients_freq_dict_all:
                ingredients_freq_dict_all[i] += 1
            else:
                ingredients_freq_dict_all[i] = 1
                
        for i in recipe_ingredients:
            if i in ingredients_measurement_dict_all:
                ingredients_measurement_dict_all[i] |= recipe_ingredients_measurement_dict[i]
            else:
                ingredients_measurement_dict_all[i] = recipe_ingredients_measurement_dict[i]
    
        for i in recipe_ingredients:
            if i in ingredients_descriptor_dict_all:
                ingredients_descriptor_dict_all[i] |= ingredients_descriptor_dict[i]
            else:
                ingredients_descriptor_dict_all[i] = ingredients_descriptor_dict[i]
                
        for i in recipe_ingredients:
            if i in ingredients_preparation_dict_all:
                ingredients_preparation_dict_all[i] |= ingredients_preparation_dict[i]
            else:
                ingredients_preparation_dict_all[i] = ingredients_preparation_dict[i]  
                
        # scraping pause to avoid being banned by site
#         time.sleep(1)
    return ingredients_freq_dict_all, ingredients_measurement_dict_all, ingredients_descriptor_dict_all, ingredients_preparation_dict_all

In [415]:
# 4 pages of vegetarian protein recipes
# 6 pages of meat recipes
# 2 pages of seafood recipes

In [439]:
%%time
vegetarian_protein_ingredients_freq_dict = dict()
vegetarian_protein_ingredients_measurement_dict = defaultdict(set)
vegetarian_protein_ingredients_descriptor_dict = defaultdict(set)
vegetarian_protein_ingredients_preparation_dict = defaultdict(set)
    
for page in range(4):
    vegetarian_protein_url = 'https://www.allrecipes.com/recipes/16778/everyday-cooking/vegetarian/protein/?page=' + str(page+1)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(vegetarian_protein_url)
    for i in ingredients_cat.keys():
        if i in vegetarian_protein_ingredients_freq_dict:
            vegetarian_protein_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            vegetarian_protein_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in vegetarian_protein_ingredients_measurement_dict:
            vegetarian_protein_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            vegetarian_protein_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in vegetarian_protein_ingredients_descriptor_dict:
            vegetarian_protein_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            vegetarian_protein_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in vegetarian_protein_ingredients_preparation_dict:
            vegetarian_protein_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            vegetarian_protein_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/218450/cilantro-lime-grilled-tofu/
1 (14 ounce) package firm tofu
quantity: 1 (14 ounce)
measurement: package
ingredient name: firm tofu
descriptor: firm
ingredient without descriptor: tofu
preparation: None

5 tablespoons chopped fresh cilantro
quantity: 5
measurement: tablespoons
ingredient name: chopped fresh cilantro
descriptor: chopped fresh
ingredient without descriptor: cilantro
preparation: None

2 teaspoons chili powder
quantity: 2
measurement: teaspoons
ingredient name: chili powder
descriptor: None
ingredient without descriptor: chili powder
preparation: None

2 cloves garlic, minced
quantity: 2
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: minced

salt and black pepper to taste
quantity: 
measurement: None
ingredient name: salt and black pepper to taste
descriptor: black
ingredient without descriptor: salt and pepper
preparation: None

1/4 cup lime juice
quantity: 1/4
measure

1 large tomato, seeded and chopped
quantity: 1
measurement: None
ingredient name: large tomato
descriptor: large
ingredient without descriptor: tomato
preparation: seeded and chopped

2 tablespoons vegetable oil
quantity: 2
measurement: tablespoons
ingredient name: vegetable oil
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

1 (14 ounce) package firm tofu, drained
quantity: 1 (14 ounce)
measurement: package
ingredient name: firm tofu
descriptor: firm
ingredient without descriptor: tofu
preparation: drained

1 teaspoon minced fresh ginger
quantity: 1
measurement: teaspoon
ingredient name: minced fresh ginger
descriptor: minced fresh
ingredient without descriptor: ginger
preparation: None

2 cloves garlic, minced
quantity: 2
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: minced

1 tablespoon sesame seeds
quantity: 1
measurement: tablespoon
ingredient name: sesame seeds
descriptor: None
ingredien

1 1/2 cups chopped green bell pepper
quantity: 1 1/2
measurement: cups
ingredient name: chopped green bell pepper
descriptor: chopped green
ingredient without descriptor: bell pepper
preparation: None

2 tablespoons corn oil
quantity: 2
measurement: tablespoons
ingredient name: corn oil
descriptor: None
ingredient without descriptor: corn oil
preparation: None

https://www.allrecipes.com/recipe/13896/tofu-parmigiana/
5 tablespoons grated Parmesan cheese
quantity: 5
measurement: tablespoons
ingredient name: grated Parmesan cheese
descriptor: grated
ingredient without descriptor: Parmesan cheese
preparation: None

1 (8 ounce) can tomato sauce
quantity: 1 (8 ounce)
measurement: can
ingredient name: tomato sauce
descriptor: None
ingredient without descriptor: tomato sauce
preparation: None

2 teaspoons dried oregano, divided
quantity: 2
measurement: teaspoons
ingredient name: dried oregano
descriptor: dried
ingredient without descriptor: oregano
preparation: divided

salt to taste
quantity

1 red onion, sliced
quantity: 1
measurement: None
ingredient name: red onion
descriptor: red
ingredient without descriptor: onion
preparation: sliced

3 tablespoons peanut oil
quantity: 3
measurement: tablespoons
ingredient name: peanut oil
descriptor: None
ingredient without descriptor: peanut oil
preparation: None

1 teaspoon cornstarch
quantity: 1
measurement: teaspoon
ingredient name: cornstarch
descriptor: None
ingredient without descriptor: cornstarch
preparation: None

1 tablespoon brown sugar
quantity: 1
measurement: tablespoon
ingredient name: brown sugar
descriptor: brown
ingredient without descriptor: sugar
preparation: None

1 green chile pepper, chopped
quantity: 1
measurement: None
ingredient name: green chile pepper
descriptor: green chile
ingredient without descriptor: pepper
preparation: chopped

3 tablespoons white vinegar
quantity: 3
measurement: tablespoons
ingredient name: white vinegar
descriptor: white
ingredient without descriptor: vinegar
preparation: None

1 t

1 bunch fresh spinach, stems removed
quantity: 1
measurement: bunch
ingredient name: fresh spinach
descriptor: fresh
ingredient without descriptor: spinach
preparation: stems removed

1 tablespoon curry paste
quantity: 1
measurement: tablespoon
ingredient name: curry paste
descriptor: None
ingredient without descriptor: curry paste
preparation: None

https://www.allrecipes.com/recipe/88768/breaded-fried-softly-spiced-tofu/
1/2 teaspoon freshly ground black pepper
quantity: 1/2
measurement: teaspoon
ingredient name: freshly ground black pepper
descriptor: freshly ground black
ingredient without descriptor: pepper
preparation: None

3 tablespoons vegetable oil
quantity: 3
measurement: tablespoons
ingredient name: vegetable oil
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

2 cups vegetable broth
quantity: 2
measurement: cups
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: None

1 (16 ounce) package extr

...

2/3 cup peanut butter
quantity: 2/3
measurement: cup
ingredient name: peanut butter
descriptor: None
ingredient without descriptor: peanut butter
preparation: None

3 cloves garlic, minced
quantity: 3
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: minced

4 cups water
quantity: 4
measurement: cups
ingredient name: water
descriptor: None
ingredient without descriptor: water
preparation: None

https://www.allrecipes.com/recipe/14095/tofu-fudge-mocha-bars/
1 (12 ounce) package silken tofu, undrained
quantity: 1 (12 ounce)
measurement: package
ingredient name: silken tofu
descriptor: silken
ingredient without descriptor: tofu
preparation: undrained

1/3 cup instant decaffeinated coffee powder
quantity: 1/3
measurement: cup
ingredient name: instant decaffeinated coffee powder
descriptor: instant decaffeinated
ingredient without descriptor: coffee powder
preparation: None

2 1/3 cups turbinado sugar
quantity: 2 1/3
measurement:

2 teaspoons Chinese black vinegar
quantity: 2
measurement: teaspoons
ingredient name: Chinese black vinegar
descriptor: Chinese black
ingredient without descriptor: vinegar
preparation: None

1 teaspoon white sugar
quantity: 1
measurement: teaspoon
ingredient name: white sugar
descriptor: white
ingredient without descriptor: sugar
preparation: None

https://www.allrecipes.com/recipe/56933/stove-top-tofu-chili/
2 stalks celery, chopped
quantity: 2
measurement: stalks
ingredient name: celery
descriptor: None
ingredient without descriptor: celery
preparation: chopped

1 teaspoon chili powder
quantity: 1
measurement: teaspoon
ingredient name: chili powder
descriptor: None
ingredient without descriptor: chili powder
preparation: None

1/2 cup onion, chopped
quantity: 1/2
measurement: cup
ingredient name: onion
descriptor: None
ingredient without descriptor: onion
preparation: chopped

2 tablespoons vegetable oil
quantity: 2
measurement: tablespoons
ingredient name: vegetable oil
descriptor:

1 (12 ounce) package firm tofu - drained, patted dry, and sliced into 4 slices
quantity: 1 (12 ounce)
measurement: package
ingredient name: firm tofu
descriptor: firm
ingredient without descriptor: tofu
preparation: drained, patted dry, and sliced into 4 slices

https://www.allrecipes.com/recipe/22657/tofu-turkey-ii/
2/3 cup water
quantity: 2/3
measurement: cup
ingredient name: water
descriptor: None
ingredient without descriptor: water
preparation: None

1 teaspoon barbeque sauce
quantity: 1
measurement: teaspoon
ingredient name: barbeque sauce
descriptor: barbeque
ingredient without descriptor: sauce
preparation: None

1/2 teaspoon prepared mustard
quantity: 1/2
measurement: teaspoon
ingredient name: prepared mustard
descriptor: prepared
ingredient without descriptor: mustard
preparation: None

2 tablespoons water
quantity: 2
measurement: tablespoons
ingredient name: water
descriptor: None
ingredient without descriptor: water
preparation: None

1 tablespoon orange jam
quantity: 1
mea

6 tablespoons soy sauce
quantity: 6
measurement: tablespoons
ingredient name: soy sauce
descriptor: None
ingredient without descriptor: soy sauce
preparation: None

https://www.allrecipes.com/recipe/18062/mock-chicken/
1 (12 ounce) package firm tofu
quantity: 1 (12 ounce)
measurement: package
ingredient name: firm tofu
descriptor: firm
ingredient without descriptor: tofu
preparation: None

https://www.allrecipes.com/recipe/13989/barbeque-seitan-and-black-bean-burritos/
1 (16 ounce) can diced tomatoes
quantity: 1 (16 ounce)
measurement: can
ingredient name: diced tomatoes
descriptor: diced
ingredient without descriptor: tomatoes
preparation: None

10 (10 inch) flour tortillas
quantity: 10 (10 inch)
measurement: flour
ingredient name: tortillas
descriptor: None
ingredient without descriptor: tortillas
preparation: None

3 tablespoons olive oil
quantity: 3
measurement: tablespoons
ingredient name: olive oil
descriptor: None
ingredient without descriptor: olive oil
preparation: None

3 cup

black pepper to taste
quantity: 
measurement: None
ingredient name: black pepper to taste
descriptor: black
ingredient without descriptor: pepper
preparation: None

https://www.allrecipes.com/recipe/68485/marinated-tofu/
1 (12 ounce) bottle barbecue sauce
quantity: 1 (12 ounce)
measurement: bottle
ingredient name: barbecue sauce
descriptor: None
ingredient without descriptor: barbecue sauce
preparation: None

1 tablespoon olive oil
quantity: 1
measurement: tablespoon
ingredient name: olive oil
descriptor: None
ingredient without descriptor: olive oil
preparation: None

1 pound tofu, plain or seasoned
quantity: 1 plain or seasoned
measurement: pound
ingredient name: tofu
descriptor: None
ingredient without descriptor: tofu
preparation: None

https://www.allrecipes.com/recipe/53329/braised-tofu/
1 (14 ounce) package firm tofu
quantity: 1 (14 ounce)
measurement: package
ingredient name: firm tofu
descriptor: firm
ingredient without descriptor: tofu
preparation: None

1 (8 ounce) can water

In [440]:
sorted(vegetarian_protein_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True)

[('tofu', 66),
 ('onion', 39),
 ('garlic', 33),
 ('salt', 33),
 ('pepper', 30),
 ('oil', 27),
 ('soy sauce', 22),
 ('water', 20),
 ('olive oil', 19),
 ('bell pepper', 17),
 ('mushrooms', 14),
 ('onions', 14),
 ('cilantro', 12),
 ('sugar', 12),
 ('sesame oil', 10),
 ('garlic powder', 10),
 ('cumin', 9),
 ('carrots', 9),
 ('tomato sauce', 9),
 ('tomatoes', 9),
 ('salt and pepper', 8),
 ('celery', 8),
 ('tempeh', 8),
 ('broth', 8),
 ('basil', 8),
 ('chili powder', 7),
 ('cayenne pepper', 7),
 ('coconut milk', 7),
 ('ginger root', 7),
 ('lime juice', 6),
 ('spinach', 6),
 ('oregano', 6),
 ('mozzarella cheese', 6),
 ('curry powder', 6),
 ('beans', 6),
 ('butter', 6),
 ('tamari', 5),
 ('soy milk', 5),
 ('clove garlic', 5),
 ('ginger', 5),
 ('barbecue sauce', 5),
 ('Cheddar cheese', 5),
 ('peppers', 5),
 ('vinegar', 5),
 ('cornstarch', 5),
 ('parsley', 5),
 ('rice', 5),
 ('honey', 5),
 ('vanilla extract', 5),
 ('zucchini', 4),
 ('coriander', 4),
 ('yeast', 4),
 ('potatoes', 4),
 ('peas', 4),


In [441]:
vegetarian_protein_ingredients_measurement_dict

defaultdict(set,
            {'chili powder': {None, 'tablespoons', 'teaspoon', 'teaspoons'},
             'cilantro': {'bunch',
              'cup',
              'sprigs',
              'tablespoon',
              'tablespoons'},
             'salt and pepper': {None},
             'olive oil': {None, 'cup', 'tablespoon', 'tablespoons'},
             'garlic': {'cloves', 'tablespoons', 'teaspoon', 'teaspoons'},
             'tofu': {'container',
              'cup',
              'cups',
              'ounces',
              'package',
              'packages',
              'pound',
              'pounds'},
             'cayenne pepper': {None, 'teaspoon', 'teaspoons'},
             'lime juice': {'cup', 'tablespoon', 'tablespoons'},
             'sesame oil': {'tablespoon',
              'tablespoons',
              'teaspoon',
              'teaspoons'},
             'soy sauce': {'cup', 'tablespoon', 'tablespoons'},
             'bell pepper': {None, 'cup', 'cups'},
             

In [425]:
%%time
meat_ingredients_freq_dict = dict()
meat_ingredients_measurement_dict = defaultdict(set)
meat_ingredients_descriptor_dict = defaultdict(set)
meat_ingredients_preparation_dict = defaultdict(set)

for index in range(200, 206):
    meat_url = 'https://www.allrecipes.com/recipes/{}/meat-and-poultry/'.format(index)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(vegetarian_protein_url)
    for i in ingredients_cat.keys():
        if i in meat_ingredients_freq_dict:
            meat_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            meat_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in meat_ingredients_measurement_dict:
            meat_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            meat_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in meat_ingredients_descriptor_dict:
            meat_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            meat_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in meat_ingredients_preparation_dict:
            meat_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            meat_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/29949/tasty-breaded-seitan/
1/2 teaspoon ground coriander
quantity: 1/2
measurement: teaspoon
ingredient name: ground coriander
descriptor: ground
ingredient without descriptor: coriander
preparation: None

2 tablespoons tamari
quantity: 2
measurement: tablespoons
ingredient name: tamari
descriptor: tamari
ingredient without descriptor: tamari
preparation: None

1 teaspoon garlic powder
quantity: 1
measurement: teaspoon
ingredient name: garlic powder
descriptor: None
ingredient without descriptor: garlic powder
preparation: None

2 tablespoons vegetable oil
quantity: 2
measurement: tablespoons
ingredient name: vegetable oil
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

1/8 teaspoon ground black pepper
quantity: 1/8
measurement: teaspoon
ingredient name: ground black pepper
descriptor: ground black
ingredient without descriptor: pepper
preparation: None

1/3 cup nutritional yeast
quantity: 1/3
measurement: cup
ingredient na

2 (12 ounce) packages extra firm tofu, drained and cubed
quantity: 2 (12 ounce)
measurement: packages
ingredient name: extra firm tofu
descriptor: extra firm
ingredient without descriptor: tofu
preparation: drained and cubed

1 cup white sugar
quantity: 1
measurement: cup
ingredient name: white sugar
descriptor: white
ingredient without descriptor: sugar
preparation: None

2 tablespoons lemon juice
quantity: 2
measurement: tablespoons
ingredient name: lemon juice
descriptor: None
ingredient without descriptor: lemon juice
preparation: None

1/4 teaspoon salt
quantity: 1/4
measurement: teaspoon
ingredient name: salt
descriptor: None
ingredient without descriptor: salt
preparation: None

1 teaspoon vanilla extract
quantity: 1
measurement: teaspoon
ingredient name: vanilla extract
descriptor: None
ingredient without descriptor: vanilla extract
preparation: None

1 (9 inch) prepared graham cracker crust
quantity: 1 (9 inch)
measurement: None
ingredient name: prepared graham cracker crust
d

8 ounces vegan sausage, cut into 1/4 inch slices (optional)
quantity: 8
measurement: ounces
ingredient name: vegan sausage(optional)
descriptor: sausage(optional
ingredient without descriptor: vegan )
preparation: cut into 1/4 inch slices

2 1/2 cups vegetable broth
quantity: 2 1/2
measurement: cups
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: None

https://www.allrecipes.com/recipe/151174/vegan-corn-bread/
1 tablespoon baking powder
quantity: 1
measurement: tablespoon
ingredient name: baking powder
descriptor: None
ingredient without descriptor: baking powder
preparation: None

1/4 cup soft silken tofu
quantity: 1/4
measurement: cup
ingredient name: soft silken tofu
descriptor: soft silken
ingredient without descriptor: tofu
preparation: None

1 cup sweetened, plain soy milk
quantity: 1
measurement: cup
ingredient name: sweetened
descriptor: sweetened
ingredient without descriptor: sweetened
preparation: plain soy milk

1/4 c

3 fresh green chile peppers
quantity: 3
measurement: None
ingredient name: fresh green chile peppers
descriptor: fresh green chile
ingredient without descriptor: peppers
preparation: None

1 small onion, quartered
quantity: 1
measurement: None
ingredient name: small onion
descriptor: small
ingredient without descriptor: onion
preparation: quartered

1 tablespoon chopped fresh cilantro
quantity: 1
measurement: tablespoon
ingredient name: chopped fresh cilantro
descriptor: chopped fresh
ingredient without descriptor: cilantro
preparation: None

8 ounces tofu, diced
quantity: 8
measurement: ounces
ingredient name: tofu
descriptor: None
ingredient without descriptor: tofu
preparation: diced

4 cloves garlic, peeled
quantity: 4
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: peeled

2 tablespoons lime juice
quantity: 2
measurement: tablespoons
ingredient name: lime juice
descriptor: None
ingredient without descriptor: lime juic

1 1/2 teaspoons minced garlic
quantity: 1 1/2
measurement: teaspoons
ingredient name: minced garlic
descriptor: minced
ingredient without descriptor: garlic
preparation: None

1/4 cup liquid amino acid (such as Bragg®)
quantity: 1/4
measurement: cup
ingredient name: liquid amino acid (such as Bragg)
descriptor: liquid
ingredient without descriptor: amino acid (such as Bragg)
preparation: None

4 cups vegetable broth
quantity: 4
measurement: cups
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: None

1 cup vital wheat gluten
quantity: 1
measurement: cup
ingredient name: vital wheat gluten
descriptor: vital gluten
ingredient without descriptor: vital wheat gluten
preparation: None

Dough:
https://www.allrecipes.com/recipe/238521/basic-seitan-wheat-meat-vegan-meat-substitute/
1/2 cup vegetable broth
quantity: 1/2
measurement: cup
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: 

...

1/4 onion, chopped
quantity: 1/4
measurement: None
ingredient name: onion
descriptor: None
ingredient without descriptor: onion
preparation: chopped

1/2 cup chopped green bell pepper
quantity: 1/2
measurement: cup
ingredient name: chopped green bell pepper
descriptor: chopped green
ingredient without descriptor: bell pepper
preparation: None

3/4 cup extra firm tofu, drained, crumbled
quantity: 3/4
measurement: cup
ingredient name: extra firm tofu
descriptor: extra firm
ingredient without descriptor: tofu
preparation: drained, crumbled

1/2 cup dry white beans, soaked overnight
quantity: 1/2
measurement: cup
ingredient name: dry white beans
descriptor: dry white
ingredient without descriptor: beans
preparation: soaked overnight

6 cups chopped fresh tomatoes
quantity: 6
measurement: cups
ingredient name: chopped fresh tomatoes
descriptor: chopped fresh
ingredient without descriptor: tomatoes
preparation: None

1/2 cup chopped red bell pepper
quantity: 1/2
measurement: cup
ingredient n

salt and pepper to taste
quantity: 
measurement: None
ingredient name: salt and pepper to taste
descriptor: None
ingredient without descriptor: salt and pepper
preparation: None

1 (16 ounce) package extra firm tofu
quantity: 1 (16 ounce)
measurement: package
ingredient name: extra firm tofu
descriptor: extra firm
ingredient without descriptor: tofu
preparation: None

3 tablespoons olive oil
quantity: 3
measurement: tablespoons
ingredient name: olive oil
descriptor: None
ingredient without descriptor: olive oil
preparation: None

1/4 cup chopped green onions
quantity: 1/4
measurement: cup
ingredient name: chopped green onions
descriptor: chopped green
ingredient without descriptor: onions
preparation: None

1/3 cup soy sauce
quantity: 1/3
measurement: cup
ingredient name: soy sauce
descriptor: None
ingredient without descriptor: soy sauce
preparation: None

3 cloves garlic, minced
quantity: 3
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: ga

4 whole wheat hamburger buns, split
quantity: 4
measurement: None
ingredient name: whole wheat hamburger buns
descriptor: whole
ingredient without descriptor: wheat hamburger buns
preparation: split

1 cup bread crumbs
quantity: 1
measurement: cup
ingredient name: bread crumbs
descriptor: None
ingredient without descriptor: bread crumbs
preparation: None

1 teaspoon kelp powder
quantity: 1
measurement: teaspoon
ingredient name: kelp powder
descriptor: None
ingredient without descriptor: kelp powder
preparation: None

olive oil, as needed
quantity: 
measurement: None
ingredient name: olive oil
descriptor: None
ingredient without descriptor: olive oil
preparation: as needed

1/4 teaspoon paprika
quantity: 1/4
measurement: teaspoon
ingredient name: paprika
descriptor: None
ingredient without descriptor: paprika
preparation: None

1/4 teaspoon onion powder or flakes
quantity: 1/4
measurement: teaspoon
ingredient name: onion powder or flakes
descriptor: None
ingredient without descriptor: o

6 tablespoons rice vinegar
quantity: 6
measurement: tablespoons
ingredient name: rice vinegar
descriptor: None
ingredient without descriptor: rice vinegar
preparation: None

8 green onions, halved lengthwise
quantity: 8
measurement: None
ingredient name: green onions
descriptor: green
ingredient without descriptor: onions
preparation: halved lengthwise

1/2 (16 ounce) package linguine pasta
quantity: 1/2 (16 ounce)
measurement: package
ingredient name: linguine pasta
descriptor: linguine
ingredient without descriptor: pasta
preparation: None

1 tablespoon sesame oil
quantity: 1
measurement: tablespoon
ingredient name: sesame oil
descriptor: None
ingredient without descriptor: sesame oil
preparation: None

2 cloves garlic, chopped
quantity: 2
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: chopped

3 tablespoons honey
quantity: 3
measurement: tablespoons
ingredient name: honey
descriptor: None
ingredient without descriptor:

5 small eggplants
quantity: 5
measurement: None
ingredient name: small eggplants
descriptor: small
ingredient without descriptor: eggplants
preparation: None

https://www.allrecipes.com/recipe/25917/vegan-chunky-chili/
1/4 onion, chopped
quantity: 1/4
measurement: None
ingredient name: onion
descriptor: None
ingredient without descriptor: onion
preparation: chopped

1/2 cup chopped green bell pepper
quantity: 1/2
measurement: cup
ingredient name: chopped green bell pepper
descriptor: chopped green
ingredient without descriptor: bell pepper
preparation: None

3/4 cup extra firm tofu, drained, crumbled
quantity: 3/4
measurement: cup
ingredient name: extra firm tofu
descriptor: extra firm
ingredient without descriptor: tofu
preparation: drained, crumbled

1/2 cup dry white beans, soaked overnight
quantity: 1/2
measurement: cup
ingredient name: dry white beans
descriptor: dry white
ingredient without descriptor: beans
preparation: soaked overnight

6 cups chopped fresh tomatoes
quantity: 6

In [426]:
sorted(meat_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True)

[('tofu', 96),
 ('oil', 48),
 ('salt', 48),
 ('olive oil', 42),
 ('onion', 42),
 ('water', 36),
 ('pepper', 30),
 ('garlic', 30),
 ('sesame oil', 24),
 ('garlic powder', 18),
 ('sugar', 18),
 ('onions', 18),
 ('tomatoes', 18),
 ('celery', 18),
 ('bell pepper', 18),
 ('beans', 18),
 ('yeast', 12),
 ('tamari', 12),
 ('broth', 12),
 ('soy sauce', 12),
 ('salt and pepper', 12),
 ('chili powder', 12),
 ('kidney beans', 12),
 ('lemon juice', 12),
 ('lentils', 12),
 ('cilantro', 12),
 ('barbecue sauce', 12),
 ('mushrooms', 12),
 ('coriander', 6),
 ('seitan', 6),
 ('cumin', 6),
 ('vital wheat gluten', 6),
 ('amino acid (such as Bragg)', 6),
 ('vinegar', 6),
 ('clove garlic', 6),
 ('kernel corn', 6),
 ('oil for frying', 6),
 ('egg', 6),
 ('Cheddar cheese', 6),
 ("confectioners' sugar", 6),
 ('banana', 6),
 ('soy milk', 6),
 ('cocoa powder', 6),
 ('cinnamon', 6),
 ('graham cracker crust', 6),
 ('vanilla extract', 6),
 ('basmati rice', 6),
 ('coconut milk', 6),
 ('curry paste', 6),
 ('wheat hambu

In [427]:
meat_ingredients_measurement_dict

defaultdict(set,
            {'coriander': {'teaspoon'},
             'yeast': {'cup', 'tablespoons'},
             'seitan': {'package'},
             'tamari': {'cup', 'tablespoons'},
             'oil': {'cup', 'tablespoon', 'tablespoons', 'teaspoons'},
             'cumin': {'teaspoon'},
             'pepper': {None, 'jalapeno', 'teaspoon'},
             'garlic powder': {None, 'teaspoon'},
             'olive oil': {None, 'cup', 'tablespoon', 'tablespoons'},
             'water': {'cup', 'cups', 'quart'},
             'vital wheat gluten': {'cup'},
             'amino acid (such as Bragg)': {'cup'},
             'garlic': {'cloves', 'teaspoons'},
             'broth': {'cups'},
             'soy sauce': {'cup', 'tablespoons'},
             'sesame oil': {'tablespoon',
              'tablespoons',
              'teaspoon',
              'teaspoons'},
             'salt and pepper': {None},
             'sugar': {'cup', 'teaspoon', 'teaspoons'},
             'vinegar': {'teaspoons'}

In [421]:
%%time
seafood_ingredients_freq_dict = dict()
seafood_ingredients_measurement_dict = defaultdict(set)
seafood_ingredients_descriptor_dict = defaultdict(set)
seafood_ingredients_preparation_dict = defaultdict(set)

for page in range(2):
    seafood_url = 'https://www.allrecipes.com/recipes/93/seafood/' + str(page+1)
    ingredients_cat, ingredients_measurement_dict_cat, ingredients_descriptor_dict_cat, ingredients_preparation_dict_cat = get_all_ingredients_in_category(vegetarian_protein_url)
    for i in ingredients_cat.keys():
        if i in seafood_ingredients_freq_dict:
            seafood_ingredients_freq_dict[i] += ingredients_cat[i]
        else:
            seafood_ingredients_freq_dict[i] = ingredients_cat[i]
    
    for i in ingredients_measurement_dict_cat.keys():
        if i in seafood_ingredients_measurement_dict:
            seafood_ingredients_measurement_dict[i] |= ingredients_measurement_dict_cat[i]
        else:
            seafood_ingredients_measurement_dict[i] = ingredients_measurement_dict_cat[i]
    
    for i in ingredients_descriptor_dict_cat.keys():
        if i in seafood_ingredients_descriptor_dict:
            seafood_ingredients_descriptor_dict[i] |= ingredients_descriptor_dict_cat[i]
        else:
            seafood_ingredients_descriptor_dict[i] = ingredients_descriptor_dict_cat[i]
    
    for i in ingredients_preparation_dict_cat.keys():
        if i in seafood_ingredients_preparation_dict:
            seafood_ingredients_preparation_dict[i] |= ingredients_preparation_dict_cat[i]
        else:
            seafood_ingredients_preparation_dict[i] = ingredients_preparation_dict_cat[i]

https://www.allrecipes.com/recipe/29949/tasty-breaded-seitan/
1/2 teaspoon ground coriander
quantity: 1/2
measurement: teaspoon
ingredient name: ground coriander
descriptor: ground
ingredient without descriptor: coriander
preparation: None

2 tablespoons tamari
quantity: 2
measurement: tablespoons
ingredient name: tamari
descriptor: tamari
ingredient without descriptor: tamari
preparation: None

1 teaspoon garlic powder
quantity: 1
measurement: teaspoon
ingredient name: garlic powder
descriptor: None
ingredient without descriptor: garlic powder
preparation: None

2 tablespoons vegetable oil
quantity: 2
measurement: tablespoons
ingredient name: vegetable oil
descriptor: vegetable
ingredient without descriptor: oil
preparation: None

1/8 teaspoon ground black pepper
quantity: 1/8
measurement: teaspoon
ingredient name: ground black pepper
descriptor: ground black
ingredient without descriptor: pepper
preparation: None

1/3 cup nutritional yeast
quantity: 1/3
measurement: cup
ingredient na

2 (12 ounce) packages extra firm tofu, drained and cubed
quantity: 2 (12 ounce)
measurement: packages
ingredient name: extra firm tofu
descriptor: extra firm
ingredient without descriptor: tofu
preparation: drained and cubed

1 cup white sugar
quantity: 1
measurement: cup
ingredient name: white sugar
descriptor: white
ingredient without descriptor: sugar
preparation: None

2 tablespoons lemon juice
quantity: 2
measurement: tablespoons
ingredient name: lemon juice
descriptor: None
ingredient without descriptor: lemon juice
preparation: None

1/4 teaspoon salt
quantity: 1/4
measurement: teaspoon
ingredient name: salt
descriptor: None
ingredient without descriptor: salt
preparation: None

1 teaspoon vanilla extract
quantity: 1
measurement: teaspoon
ingredient name: vanilla extract
descriptor: None
ingredient without descriptor: vanilla extract
preparation: None

1 (9 inch) prepared graham cracker crust
quantity: 1 (9 inch)
measurement: None
ingredient name: prepared graham cracker crust
d

2 1/2 cups vegetable broth
quantity: 2 1/2
measurement: cups
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: None

https://www.allrecipes.com/recipe/151174/vegan-corn-bread/
1 tablespoon baking powder
quantity: 1
measurement: tablespoon
ingredient name: baking powder
descriptor: None
ingredient without descriptor: baking powder
preparation: None

1/4 cup soft silken tofu
quantity: 1/4
measurement: cup
ingredient name: soft silken tofu
descriptor: soft silken
ingredient without descriptor: tofu
preparation: None

1 cup sweetened, plain soy milk
quantity: 1
measurement: cup
ingredient name: sweetened
descriptor: sweetened
ingredient without descriptor: sweetened
preparation: plain soy milk

1/4 cup turbinado sugar
quantity: 1/4
measurement: cup
ingredient name: turbinado sugar
descriptor: None
ingredient without descriptor: turbinado sugar
preparation: None

1 cup all-purpose flour
quantity: 1
measurement: cup
ingredient name: all-

3 fresh green chile peppers
quantity: 3
measurement: None
ingredient name: fresh green chile peppers
descriptor: fresh green chile
ingredient without descriptor: peppers
preparation: None

1 small onion, quartered
quantity: 1
measurement: None
ingredient name: small onion
descriptor: small
ingredient without descriptor: onion
preparation: quartered

1 tablespoon chopped fresh cilantro
quantity: 1
measurement: tablespoon
ingredient name: chopped fresh cilantro
descriptor: chopped fresh
ingredient without descriptor: cilantro
preparation: None

8 ounces tofu, diced
quantity: 8
measurement: ounces
ingredient name: tofu
descriptor: None
ingredient without descriptor: tofu
preparation: diced

4 cloves garlic, peeled
quantity: 4
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: peeled

2 tablespoons lime juice
quantity: 2
measurement: tablespoons
ingredient name: lime juice
descriptor: None
ingredient without descriptor: lime juic

1 1/2 teaspoons minced garlic
quantity: 1 1/2
measurement: teaspoons
ingredient name: minced garlic
descriptor: minced
ingredient without descriptor: garlic
preparation: None

1/4 cup liquid amino acid (such as Bragg®)
quantity: 1/4
measurement: cup
ingredient name: liquid amino acid (such as Bragg)
descriptor: liquid
ingredient without descriptor: amino acid (such as Bragg)
preparation: None

4 cups vegetable broth
quantity: 4
measurement: cups
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: None

1 cup vital wheat gluten
quantity: 1
measurement: cup
ingredient name: vital wheat gluten
descriptor: vital gluten
ingredient without descriptor: vital wheat gluten
preparation: None

Dough:
https://www.allrecipes.com/recipe/238521/basic-seitan-wheat-meat-vegan-meat-substitute/
1/2 cup vegetable broth
quantity: 1/2
measurement: cup
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: 

1 (14 ounce) package firm water-packed tofu, drained and cubed
quantity: 1 (14 ounce)
measurement: package
ingredient name: firm water-packed tofu
descriptor: firm water-packed
ingredient without descriptor: tofu
preparation: drained and cubed

3 tablespoons sesame oil
quantity: 3
measurement: tablespoons
ingredient name: sesame oil
descriptor: None
ingredient without descriptor: sesame oil
preparation: None

1/4 teaspoon salt
quantity: 1/4
measurement: teaspoon
ingredient name: salt
descriptor: None
ingredient without descriptor: salt
preparation: None

2 tablespoons green curry paste
quantity: 2
measurement: tablespoons
ingredient name: green curry paste
descriptor: green
ingredient without descriptor: curry paste
preparation: None

1 1/2 cups water
quantity: 1 1/2
measurement: cups
ingredient name: water
descriptor: None
ingredient without descriptor: water
preparation: None

1 (10 ounce) can coconut milk
quantity: 1 (10 ounce)
measurement: can
ingredient name: coconut milk
descript

2 1/2 cups vegetable broth
quantity: 2 1/2
measurement: cups
ingredient name: vegetable broth
descriptor: vegetable
ingredient without descriptor: broth
preparation: None

https://www.allrecipes.com/recipe/151174/vegan-corn-bread/
1 tablespoon baking powder
quantity: 1
measurement: tablespoon
ingredient name: baking powder
descriptor: None
ingredient without descriptor: baking powder
preparation: None

1/4 cup soft silken tofu
quantity: 1/4
measurement: cup
ingredient name: soft silken tofu
descriptor: soft silken
ingredient without descriptor: tofu
preparation: None

1 cup sweetened, plain soy milk
quantity: 1
measurement: cup
ingredient name: sweetened
descriptor: sweetened
ingredient without descriptor: sweetened
preparation: plain soy milk

1/4 cup turbinado sugar
quantity: 1/4
measurement: cup
ingredient name: turbinado sugar
descriptor: None
ingredient without descriptor: turbinado sugar
preparation: None

1 cup all-purpose flour
quantity: 1
measurement: cup
ingredient name: all-

3 fresh green chile peppers
quantity: 3
measurement: None
ingredient name: fresh green chile peppers
descriptor: fresh green chile
ingredient without descriptor: peppers
preparation: None

1 small onion, quartered
quantity: 1
measurement: None
ingredient name: small onion
descriptor: small
ingredient without descriptor: onion
preparation: quartered

1 tablespoon chopped fresh cilantro
quantity: 1
measurement: tablespoon
ingredient name: chopped fresh cilantro
descriptor: chopped fresh
ingredient without descriptor: cilantro
preparation: None

8 ounces tofu, diced
quantity: 8
measurement: ounces
ingredient name: tofu
descriptor: None
ingredient without descriptor: tofu
preparation: diced

4 cloves garlic, peeled
quantity: 4
measurement: cloves
ingredient name: garlic
descriptor: None
ingredient without descriptor: garlic
preparation: peeled

2 tablespoons lime juice
quantity: 2
measurement: tablespoons
ingredient name: lime juice
descriptor: None
ingredient without descriptor: lime juic

In [422]:
sorted(seafood_ingredients_freq_dict.items(), key=lambda pair: pair[1], reverse=True)

[('tofu', 32),
 ('oil', 16),
 ('salt', 16),
 ('olive oil', 14),
 ('onion', 14),
 ('water', 12),
 ('pepper', 10),
 ('garlic', 10),
 ('sesame oil', 8),
 ('garlic powder', 6),
 ('sugar', 6),
 ('onions', 6),
 ('tomatoes', 6),
 ('celery', 6),
 ('bell pepper', 6),
 ('beans', 6),
 ('yeast', 4),
 ('tamari', 4),
 ('broth', 4),
 ('soy sauce', 4),
 ('salt and pepper', 4),
 ('chili powder', 4),
 ('kidney beans', 4),
 ('lemon juice', 4),
 ('lentils', 4),
 ('cilantro', 4),
 ('barbecue sauce', 4),
 ('mushrooms', 4),
 ('coriander', 2),
 ('seitan', 2),
 ('cumin', 2),
 ('vital wheat gluten', 2),
 ('amino acid (such as Bragg)', 2),
 ('vinegar', 2),
 ('clove garlic', 2),
 ('kernel corn', 2),
 ('oil for frying', 2),
 ('egg', 2),
 ('Cheddar cheese', 2),
 ("confectioners' sugar", 2),
 ('banana', 2),
 ('soy milk', 2),
 ('cocoa powder', 2),
 ('cinnamon', 2),
 ('graham cracker crust', 2),
 ('vanilla extract', 2),
 ('basmati rice', 2),
 ('coconut milk', 2),
 ('curry paste', 2),
 ('wheat hamburger buns', 2),
 ('m

In [442]:
ingredients_categorized = {}
ingredients_categorized['vegetarian_protein'] = {}
ingredients_categorized['vegetarian_protein']['ingredients'] = vegetarian_protein_ingredients_freq_dict
ingredients_categorized['vegetarian_protein']['measurement'] = vegetarian_protein_ingredients_measurement_dict
ingredients_categorized['vegetarian_protein']['descriptor'] = vegetarian_protein_ingredients_descriptor_dict
ingredients_categorized['vegetarian_protein']['preparation'] = vegetarian_protein_ingredients_preparation_dict
ingredients_categorized['meat'] = {}
ingredients_categorized['meat']['ingredients'] = meat_ingredients_freq_dict
ingredients_categorized['meat']['measurement'] = meat_ingredients_measurement_dict
ingredients_categorized['meat']['descriptor'] = meat_ingredients_descriptor_dict
ingredients_categorized['meat']['preparation'] = meat_ingredients_preparation_dict
ingredients_categorized['seafood'] = {}
ingredients_categorized['seafood']['ingredients'] = seafood_ingredients_freq_dict
ingredients_categorized['seafood']['measurement'] = seafood_ingredients_measurement_dict
ingredients_categorized['seafood']['descriptor'] = seafood_ingredients_descriptor_dict
ingredients_categorized['seafood']['preparation'] = seafood_ingredients_preparation_dict

In [443]:
# save scraped data
with open('ingredients_categorized.pickle', 'wb') as file:
    pickle.dump(ingredients_categorized, file, protocol=pickle.HIGHEST_PROTOCOL)
file.closed

True

In [444]:
# check
with open('ingredients_categorized.pickle', 'rb') as file:
    ingredients_categorized = pickle.load(file)
            
ingredients_categorized['vegetarian_protein']['descriptor'].keys()

dict_keys(['chili powder', 'cilantro', 'salt and pepper', 'olive oil', 'garlic', 'tofu', 'cayenne pepper', 'lime juice', 'sesame oil', 'soy sauce', 'bell pepper', 'mushrooms', 'pepper', 'onion', 'sriracha chili garlic sauce', 'zucchini', 'coriander', 'yeast', 'seitan', 'tamari', 'oil', 'cumin', 'garlic powder', 'soy milk', 'vegan mayonnaise', 'potatoes', 'peas', 'soy cheese', 'vegan cream cheese (such as Tofutti)', 'tomato', 'seasoning', 'clove garlic', 'vegetarian ground beef substitute', 'salt', 'celery', 'carrots', 'sesame seeds', 'mirin (sweetened rice wine)', 'ginger', 'rice vinegar', 'tempeh', 'rolls', 'barbecue sauce', 'Cheddar cheese', '1% milk', '9 inch pie crust', 'cheese', 'spinach', 'oregano', 'pepper (optional)', 'broth', 'taco seasoning mix', 'virgin olive oil', 'peppers', 'corn oil', 'Parmesan cheese', 'tomato sauce', 'bread crumbs', 'mozzarella cheese', 'basil', 'paste', 'curry powder', 'sugar', '(plum) tomatoes', 'bok choy', 'coconut milk', 'onions', 'smoke flavoring',